In [2]:
import os
from Bio import SeqIO, Seq
import pandas as pd

In [71]:
# read the xlsx file and create DataFrame
uni_file = 'uniprot-filtered-organism__Bacillus+subtilis+(strain+168)+[224308]%2--.xlsx'
assert(os.path.exists(uni_file)) #  sanity check
df_prot = pd.read_excel(uni_file)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [36]:
# print the DF head
df_prot.head()

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Transmembrane,Sequence,Gene names (primary )
0,P94485,YNAG_BACSU,reviewed,Uncharacterized protein YnaG,ynaG BSU17550,Bacillus subtilis (strain 168),91.0,"TRANSMEM 6..26; /note=""Helical""; /evidence=""...",MNVKKAAAVFSITIPIISAILIINFFTGFMSIPWQGMPVFFPLLLS...,ynaG
1,O32014,YQZH_BACSU,reviewed,Uncharacterized protein YqzH,yqzH BSU23720,Bacillus subtilis (strain 168),68.0,NaN,MEKFVEKMLGQALRQYGRNVAIDPLSPYEKQSLKAALQERRNEEPD...,yqzH
2,O34838,YOPW_BACSU,reviewed,SPbeta prophage-derived uncharacterized protei...,yopW BSU20740,Bacillus subtilis (strain 168),111.0,NaN,MELSLDELKLCLKPLVFFGELKLEISDYEEGKKIEVLDHDEGSLIN...,yopW
3,O34460,YTRI_BACSU,reviewed,Sporulation membrane protein YtrI,ytrI BSU29240,Bacillus subtilis (strain 168),167.0,"TRANSMEM 15..35; /note=""Helical""; /evidence=...",MRVPQHYKKPGWQRFFAGMMCGAVISWFFFLFTYGTFQEEQVSLIE...,ytrI
4,O34702,YOTC_BACSU,reviewed,SPbeta prophage-derived uncharacterized protei...,yotC yokC BSU19930,Bacillus subtilis (strain 168),63.0,NaN,MENEVVFFCRKCNHHLFAKNPMINTLKVISEMDCPNCGEEGYHNWI...,yotC


In [72]:
# # find all the protiens' gene name
# prot_names = [x.split(" ")[0] for x in df_prot['Gene names'].apply(str)]
# print(prot_names[0:5])
# df_prot.drop(columns= ,inplace=True)
uni_prot = df_prot['Gene names  (primary )']
uni_prot = uni_prot.dropna()

prot_names = list(uni_prot)

print(len(prot_names))

6085


In [ ]:
# df_prot['pos'] = df_prot['Gene names'].str.find(' ')
# df_prot['names'] = df_prot.apply(lambda x: x['Gene names'][0:df_prot['pos']] if x['Gene names'] != "" else None ,axis=1)
# del df_prot['pos'] 
# df_prot

In [20]:
## read file and import details
gene_bank_file = 'BS168.gb'
 # making sure that the path is valid
assert(os.path.exists(gene_bank_file))

with open(gene_bank_file, "r") as input_handle:
    record_gb = SeqIO.read(input_handle, "genbank")
    
features = record_gb.features
print(type(features))

print(features[2])

<class 'list'>
type: CDS
location: [409:1750](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['EnsemblGenomes-Gn:BSU00010', 'EnsemblGenomes-Tr:CAB11777', 'GOA:P05648', 'InterPro:IPR001957', 'InterPro:IPR003593', 'InterPro:IPR010921', 'InterPro:IPR013159', 'InterPro:IPR013317', 'InterPro:IPR018312', 'InterPro:IPR020591', 'InterPro:IPR024633', 'InterPro:IPR027417', 'PDB:4TPS', 'SubtiList:BG10065', 'UniProtKB/Swiss-Prot:P05648', 'GeneID:939978']
    Key: experiment, Value: ['publication(s) with functional evidences, PMID:2167836, 2846289, 12682299, 16120674, 1779750, 28166228']
    Key: function, Value: ['16.9: Replicate']
    Key: gene, Value: ['dnaA']
    Key: locus_tag, Value: ['BSU_00010']
    Key: note, Value: ['Evidence 1a: Function from experimental evidences in the studied strain; PubMedId: 2167836, 2846289, 12682299, 16120674, 1779750, 28166228; Product type f : factor']
    Key: old_locus_tag, Value: ['BSU00010']
    Key: product, Value: ['chromosomal

In [81]:
# find all CDS in GeneBank file
features_prot = []

for feature in features:
    if feature.type == 'CDS' and feature.qualifiers.get('gene') is not None:
        features_prot.append(feature.qualifiers.get('gene')[0])

print(features_prot[0])
print(len(features_prot))

dnaA
4177


In [60]:
# compare lists and create 3 lists

# same protien in both
same_prot = set(features_prot).intersection(set(prot_names))
# protien exist only in genebank file 
gb_only = set(features_prot).difference(set(prot_names))
# protien exist only in Uniport file 
uni_only = set(prot_names).difference(set(features_prot))

uni_only = pd.Series(list(uni_only))
uni_same = list(same_prot)
print('Same protiens in both files: ', len(same_prot))
print('Protiens in GeneBank only: ', len(gb_only))
print('Protiens in UniProt only: ', len(uni_only))  

# print(list(same_prot)[0:5])

Same protiens in both files:  3714
Protiens in GeneBank only:  457
Protiens in UniProt only:  769


check where is the difference come from

In [50]:
# check protien in GB  
gb_same = []
gb_diff = []

for feature in features:
    if feature.type == 'CDS':
        if feature.qualifiers.get('gene') is not None:
            if feature.qualifiers.get('gene')[0] in list(same_prot):
                gb_same.append(feature)
            else:
                gb_diff.append(feature)

print(len(gb_same), len(gb_diff))

3720 457


try to get datafram from the original DF so the name is in the gene name list "prot name" above.

In [83]:
# Data frame of the differences, e.g. gene only in UniPort.
df_diff = df_prot[~df_prot['Gene names  (primary )'].isin(uni_same)]

df_diff.count()


Entry                     3390
Entry name                3390
Status                    3390
Protein names             3390
Gene names                3389
Organism                  3390
Length                    3390
Transmembrane              832
Sequence                  3390
Gene names  (primary )     934
dtype: int64